In [1]:
import pandas as pd
import re
import json
import ast
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from Levenshtein import distance as lev
import time


In [8]:
food_path = "../data/dev_set.json"
flavor_path = "../data/flavor_DB.json"

In [9]:
def flatten(path, cols, meta):
   # load data using Python JSON module
    with open(path,'r') as f:
        data = json.loads(f.read())
        
    # Flatten data
    df = pd.json_normalize(
        data, 
        record_path =cols, 
        meta=meta
    )
    return df


In [10]:
food_df = pd.read_json(food_path)
food_df.info()
food_df.iloc[0:5, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   fsa_lights_per100g   1000 non-null   object
 1   id                   1000 non-null   object
 2   ingredients          1000 non-null   object
 3   instructions         1000 non-null   object
 4   nutr_per_ingredient  1000 non-null   object
 5   nutr_values_per100g  1000 non-null   object
 6   partition            1000 non-null   object
 7   quantity             1000 non-null   object
 8   title                1000 non-null   object
 9   unit                 1000 non-null   object
 10  url                  1000 non-null   object
 11  weight_per_ingr      1000 non-null   object
dtypes: object(12)
memory usage: 93.9+ KB


,fsa_lights_per100g,id,ingredients,instructions,nutr_per_ingredient,nutr_values_per100g,partition,quantity,title,unit,url,weight_per_ingr
0,"{'fat': 'red', 'salt': 'green', 'saturates': '...",fb51dbcb2f,"[{'text': 'butter, without salt'}, {'text': 's...","[{'text': 'Finely chop the dry apricots.'}, {'...","[{'fat': 48.666000000000004, 'nrg': 430.2, 'pr...","{'energy': 493.17972350230417, 'fat': 33.46962...",test,"[{'text': '60'}, {'text': '1/3'}, {'text': '2'...",Apricot Florentines Recipe,"[{'text': 'g'}, {'text': 'cup'}, {'text': 'tab...",http://cookeatshare.com/recipes/apricot-floren...,"[60.0, 48.0, 15.625, 71.5, 32.5, 125.0]"
1,"{'fat': 'red', 'salt': 'orange', 'saturates': ...",46dbd27b9a,"[{'text': 'spices, poppy seed'}, {'text': 'vin...",[{'text': 'Combine ingredients in a blender.'}...,"[{'fat': 3.48, 'nrg': 45.0, 'pro': 1.5, 'sat':...","{'energy': 349.68761484748256, 'fat': 27.34962...",train,"[{'text': '1'}, {'text': '3/4'}, {'text': '1/2...",Poppyseed Salad Dressing,"[{'text': 'tablespoon'}, {'text': 'cup'}, {'te...",http://tastykitchen.com/recipes/salads/poppyse...,"[8.4, 178.8, 100.80000000000001, 2.4, 6.0, 3.7..."
2,"{'fat': 'red', 'salt': 'orange', 'saturates': ...",dce5271a52,"[{'text': 'apples, raw, with skin'}, {'text': ...","[{'text': 'Toss all ingredients in bowl.'}, {'...","[{'fat': 2.52, 'nrg': 780.0, 'pro': 3.96, 'sat...","{'energy': 292.1303988638368, 'fat': 28.858143...",test,"[{'text': '12'}, {'text': '13'}, {'text': '14'...",Country Style Waldorf Salad for 2,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/country-style-waldo...,"[1500.0, 1313.0, 2114.0, 14.625, 47.0, 3091.20..."
3,"{'fat': 'red', 'salt': 'green', 'saturates': '...",8f873f5a14,"[{'text': 'butter, without salt'}, {'text': 'w...",[{'text': 'Mix butter and flour together well....,"[{'fat': 2211.84, 'nrg': 19584.0, 'pro': 23.04...","{'energy': 674.6799974527162, 'fat': 73.404763...",train,"[{'text': '12'}, {'text': '1'}, {'text': '2'},...",Patti's Yummy Cheese Nuggets,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 'c...",http://www.food.com/recipe/pattis-yummy-cheese...,"[2726.3999999999996, 125.0, 264.0, 25.2]"
4,"{'fat': 'orange', 'salt': 'red', 'saturates': ...",b9b89434c6,"[{'text': 'beans, snap, green, raw'}, {'text':...","[{'text': 'Preheat oven to 400 degrees F.'}, {...","[{'fat': 0.44, 'nrg': 62.0, 'pro': 3.66, 'sat'...","{'energy': 138.38721664808622, 'fat': 10.88814...",train,"[{'text': '2'}, {'text': '1/4'}, {'text': '2'}...",Asian Green Beans,"[{'text': 'cup'}, {'text': 'cup'}, {'text': 't...",http://tastykitchen.com/recipes/sidedishes/asi...,"[200.0, 72.0, 27.0, 19.5, 17.875]"


In [11]:
## read file
flavor_df = pd.read_json(flavor_path);
flavor_df.info()
flavor_df.iloc[:5, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   category               934 non-null    object
 1   entity_id              934 non-null    int64 
 2   category_readable      934 non-null    object
 3   entity_alias_basket    934 non-null    object
 4   entity_alias_readable  934 non-null    object
 5   natural_source_name    934 non-null    object
 6   entity_alias           934 non-null    object
 7   molecules              934 non-null    object
 8   natural_source_url     934 non-null    object
 9   entity_alias_url       934 non-null    object
 10  entity_alias_synonyms  934 non-null    object
dtypes: int64(1), object(10)
memory usage: 80.4+ KB


,category,entity_id,category_readable,entity_alias_basket,entity_alias_readable,natural_source_name,entity_alias,molecules,natural_source_url,entity_alias_url,entity_alias_synonyms
0,animalproduct,0,Animal Product,"egg, egg-boiled, egg-cooked, egg-scrambled",Egg,Chicken,egg,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Chicken,https://en.wikipedia.org/wiki/Egg_as_food,Egg
1,bakery,1,Bakery,bakery-products,Bakery Products,Poacceae,bakery,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bakery,Bakery Products
2,bakery,2,Bakery,"bread, bread-preferment",Bread,Poacceae,bread,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Poaceae,https://en.wikipedia.org/wiki/Bread,Bread
3,bakery,3,Bakery,bread-rye,Rye Bread,Rye,bread-rye,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Rye,https://en.wikipedia.org/wiki/Rye_bread,Rye Bread
4,bakery,4,Bakery,bread-wheaten,Wheaten Bread,Wheat,bread-wheaten,"[{'bond_stereo_count': 0, 'undefined_atom_ster...",https://en.wikipedia.org/wiki/Wheat,https://en.wikipedia.org/wiki/Soda_bread,"Soda scones, Soda farls"


In [12]:
## flatten molecules from nested array
flatten_flavor_df = flatten(flavor_path, ["molecules"], ["entity_id", "category", "entity_alias_readable"])
flatten_flavor_df.iloc[0:3,:]

,bond_stereo_count,undefined_atom_stereocenter_count,taste,functional_groups,fooddb_flavor_profile,supersweetdb_id,fema_number,fooddb_id,common_name,hba_count,...,hbd_count,fema_flavor_profile,complexity,heavy_atom_count,defined_atom_stereocenter_count,monoisotopic_mass,atom_stereo_count,entity_id,category,entity_alias_readable
0,0,0,,amine@primary amine@primary aliphatic amine (a...,faint,,3819,FDB002257,L-arginine,4,...,4,,176.0,12,1,174.112,1,0,animalproduct,Egg
1,0,0,"warm, over-ripe, fruity flavor below 0.1 ppm",aromatic compound@heterocyclic compound,very strong@mothball@animal@fecal@indole@civet,,3019,FDB010993,3-Methylindole,0,...,1,,122.0,10,0,131.073,0,0,animalproduct,Egg
2,0,0,,aromatic compound@heterocyclic compound,roast beef@roasted nuts@grass@medical@cocoa@wo...,,3272,FDB013954,"2,5-Dimethylpyrazine",2,...,0,"Cocoa, Roast Beef, Roasted Nut",62.9,8,0,108.069,0,0,animalproduct,Egg


In [13]:
## flatten ingredients from nested array
partition = 200
flatten_food_df = flatten(food_path, ["ingredients"], ["id", "title"])
# flatten_food_df = flatten_food_df.iloc[0:partition, :] # partition to a small subset for testing

## re-order the column header and rename ingredient
flatten_food_df = flatten_food_df[["id", "title", "text"]]
flatten_food_df = flatten_food_df.rename(columns = {"text": "ingredient"})
flatten_food_df = flatten_food_df.drop_duplicates(subset=['id', 'ingredient']) # remove duplicated ingredients for each food
flatten_food_df = flatten_food_df.reset_index(drop=True)

In [14]:
flatten_food_df.info()
flatten_food_df.iloc[0:12, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5878 entries, 0 to 5877
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          5878 non-null   object
 1   title       5878 non-null   object
 2   ingredient  5878 non-null   object
dtypes: object(3)
memory usage: 137.9+ KB


,id,title,ingredient
0,fb51dbcb2f,Apricot Florentines Recipe,"butter, without salt"
1,fb51dbcb2f,Apricot Florentines Recipe,"sugars, brown"
2,fb51dbcb2f,Apricot Florentines Recipe,"wheat flour, white, all-purpose, unenriched"
3,fb51dbcb2f,Apricot Florentines Recipe,"nuts, almonds"
4,fb51dbcb2f,Apricot Florentines Recipe,"apricots, dried, sulfured, uncooked"
5,fb51dbcb2f,Apricot Florentines Recipe,"spartan, real semi-sweet chocolate baking chip..."
6,46dbd27b9a,Poppyseed Salad Dressing,"spices, poppy seed"
7,46dbd27b9a,Poppyseed Salad Dressing,"vinegar, cider"
8,46dbd27b9a,Poppyseed Salad Dressing,"sugars, granulated"
9,46dbd27b9a,Poppyseed Salad Dressing,"spices, onion powder"


In [15]:
## count number of uninque ingredient for each food
## after deduplicated the ingredients, group them into list
unique_ingredient = flatten_food_df[["id", "ingredient"]].groupby(["id"]).agg(unique_ingredient=("ingredient", lambda x : list(x))).reset_index()
unique_ingredient["ingredient_count"] = unique_ingredient["unique_ingredient"].apply(lambda x : len(x))

In [16]:
unique_ingredient.info()
unique_ingredient.iloc[:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1000 non-null   object
 1   unique_ingredient  1000 non-null   object
 2   ingredient_count   1000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


,id,unique_ingredient,ingredient_count
0,005e671ac4,"[cream, fluid, heavy whipping, sugars, brown, ...",5
1,00703db954,"[oats, seeds, sesame seeds, whole, dried, seed...",8
2,007e544c80,"[soup, chicken broth or bouillon, dry, onions,...",9
3,00a0ea8691,"[rice, white, long-grain, regular, unenriched,...",9
4,00deb3f9f4,"[oil, olive, salad or cooking, vinegar, balsam...",9
5,00fb62a5c5,"[strawberries, raw, sugars, granulated, spices...",4
6,01680dde21,"[wheat flour, white, all-purpose, unenriched, ...",9
7,01e8a5eafa,"[cheese, parmesan, hard, candies, marshmallows...",4
8,021fc34777,"[cheese, cottage, creamed, large or small curd...",3
9,02ac814cc8,"[pork, fresh, loin, country-style ribs, separa...",10


In [21]:
## flatten, extract the molecule info
flatten_flavor_df = flatten_flavor_df[["category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name", "taste", "odor", "flavor_profile"]]
flatten_flavor_df.iloc[0:10, :]

,category,entity_alias_readable,fooddb_id,pubchem_id,common_name,iupac_name,taste,odor,flavor_profile
0,animalproduct,Egg,FDB002257,6322,L-arginine,(2S)-2-amino-5-(diaminomethylideneamino)pentan...,,,faint
1,animalproduct,Egg,FDB010993,6736,3-Methylindole,3-methyl-1H-indole,"warm, over-ripe, fruity flavor below 0.1 ppm","characteristic fecal odor at high levels, beco...",very strong@mothball@animal@fecal@indole@civet
2,animalproduct,Egg,FDB013954,31252,"2,5-Dimethylpyrazine","2,5-dimethylpyrazine",,,roast beef@roasted nuts@grass@medical@cocoa@wo...
3,animalproduct,Egg,FDB008174,7909,4-Methyl-2-pentanone,4-methylpentan-2-one,,"pleasant odor@faint, ketonic and camphor odor",solvent@spice@herbal@sharp@fruity@dairy@green
4,animalproduct,Egg,FDB008127,7284,2-Methylbutyraldehyde,2-methylbutanal,,,musty@coffee@cocoa@nutty@almond
5,animalproduct,Egg,FDB012552,7501,Styrene,styrene,,"extremely penetrating@an aromatic odor@sweet, ...",balsamic@gasoline@floral@sweet@balsam@plastic
6,animalproduct,Egg,FDB008326,9609,Diethyl sulfide,ethylsulfanylethane,,garlic-like odor@ethereal odor,ethereal@sulfurous
7,animalproduct,Egg,FDB012084,12180,Methyl butyrate,methyl butanoate,"apple-like, sweet taste@not very powerful@belo...",apple-like odor,apple@pineapple@ether@banana@fruit@sweet@fruity
8,animalproduct,Egg,FDB003349,18827,1-Octen-3-Ol,oct-1-en-3-ol,,,raw@fishy@oily@earthy@fungal@chicken@mushroom@...
9,animalproduct,Egg,FDB008541,18635,3-(Methylthio)propionaldehyde,3-methylsulfanylpropanal,,,potato@vegetable@tomato@fishy@musty@earthy@cre...


In [18]:
## fuzzywuzzy implementation
ingredients = flavor_df["entity_alias_readable"].unique()
start = time.process_time()
i = 2 ## pick first two key words
flatten_food_df["key"] = flatten_food_df['ingredient'].apply(lambda x : [process.extract(','.join(x.split(',')[:i]), ingredients, limit=1)][0][0][0])
duration = time.process_time() - start 
print(duration)    

266.952291


In [ ]:
# # levenshtein implementation
# ingredients = flatten_flavor_df["entity_alias_readable"].unique()
# start = time.process_time()
# i = 2
# flatten_food_df['key']=flatten_food_df['ingredient'].apply(lambda x : min(ingredients, key=lambda j : lev(','.join(x.split(',')[:i]), j))) 
# duration = time.process_time() - start 
# print(duration)  

In [22]:
flatten_food_df.iloc[0:10, :]

,id,title,ingredient,key
0,fb51dbcb2f,Apricot Florentines Recipe,"butter, without salt",Butter
1,fb51dbcb2f,Apricot Florentines Recipe,"sugars, brown",Sugar
2,fb51dbcb2f,Apricot Florentines Recipe,"wheat flour, white, all-purpose, unenriched",Wheat
3,fb51dbcb2f,Apricot Florentines Recipe,"nuts, almonds",Almond
4,fb51dbcb2f,Apricot Florentines Recipe,"apricots, dried, sulfured, uncooked",Apricot
5,fb51dbcb2f,Apricot Florentines Recipe,"spartan, real semi-sweet chocolate baking chip...",Chocolate
6,46dbd27b9a,Poppyseed Salad Dressing,"spices, poppy seed",Poppy Seed
7,46dbd27b9a,Poppyseed Salad Dressing,"vinegar, cider",Apple Cider Vinegar
8,46dbd27b9a,Poppyseed Salad Dressing,"sugars, granulated",Sugar
9,46dbd27b9a,Poppyseed Salad Dressing,"spices, onion powder",Onion


In [23]:
## join two dataset on ingredients
merged_df = flatten_flavor_df.merge(flatten_food_df,left_on='entity_alias_readable',right_on='key', how="inner")
merged_df = merged_df.sort_values(by=["id"])
merged_df = merged_df.reset_index(drop=True)
## reorder the cols
merged_df = merged_df[["id", "title", "ingredient", "category", "entity_alias_readable", "fooddb_id", "pubchem_id", "common_name", "iupac_name", "taste", "odor", "flavor_profile"]]
merged_df = merged_df.rename(columns = {"title": "food", "entity_alias_readable": "ingredient_flavorDB"})


In [15]:
## flatten col "nutrition per 100g"
## then join
# with open(food_path,'r') as f:
#     data = json.loads(f.read())
# nutr = pd.json_normalize(data)
# flatten_nutr = nutr.filter(regex="nutr_values_per100g")
# flatten_nutr.insert(0, "title", nutr["title"], True)
# flatten_nutr.insert(0, "id", nutr["id"], True)

In [24]:
merged_df.info()
merged_df.iloc[0:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584128 entries, 0 to 584127
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   584128 non-null  object
 1   food                 584128 non-null  object
 2   ingredient           584128 non-null  object
 3   category             584128 non-null  object
 4   ingredient_flavorDB  584128 non-null  object
 5   fooddb_id            584128 non-null  object
 6   pubchem_id           584128 non-null  int64 
 7   common_name          584128 non-null  object
 8   iupac_name           584128 non-null  object
 9   taste                584128 non-null  object
 10  odor                 584128 non-null  object
 11  flavor_profile       584128 non-null  object
dtypes: int64(1), object(11)
memory usage: 53.5+ MB


,id,food,ingredient,category,ingredient_flavorDB,fooddb_id,pubchem_id,common_name,iupac_name,taste,odor,flavor_profile
0,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB009020,247,betaine,2-(trimethylazaniumyl)acetate,sweet,,bland
1,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB003285,11552,3-Methylbutanal,3-methylbutanal,"at very low levels the flavor is warm, herbace...","apple-like odor@powerful penetrating, acrid odor",peach@sour@chocolate@ethereal@malt@fatty@aldeh...
2,005e671ac4,Pecan Pralines,"butter, without salt",dairy,Butter,FDB012552,7501,Styrene,styrene,,"extremely penetrating@an aromatic odor@sweet, ...",balsamic@gasoline@floral@sweet@balsam@plastic
3,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB020874,9895,beta-Cyclocitral,"2,6,6-trimethylcyclohexene-1-carbaldehyde",,,mint@saffron@tobacco@herbal@damascone@sweet@tr...
4,005e671ac4,Pecan Pralines,"butter, without salt",dairy,Butter,FDB001224,644104,CID 644104,"(2S)-2-[(1R)-1,2-dihydroxyethyl]-4,5-dihydroxy...",,,grassy@very mild
5,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB012090,7519,Anisole,anisole,sweet,sweet anise-like odor@agreeable aromatic odor@...,butter@ethereal@alcohol@gasoline@anise@cheese@...
6,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB008173,11508,3-Methyl-1-Pentanol,3-methylpentan-1-ol,,,wine@fusel@cocoa@fruity@cognac@green
7,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB000881,13144,calcium lactate,calcium;2-hydroxypropanoate,,,odorless
8,005e671ac4,Pecan Pralines,"butter, without salt",dairy,Butter,FDB012084,12180,Methyl butyrate,methyl butanoate,"apple-like, sweet taste@not very powerful@belo...",apple-like odor,apple@pineapple@ether@banana@fruit@sweet@fruity
9,005e671ac4,Pecan Pralines,vanilla extract,fruit-essence,Vanilla,FDB013794,1549026,Geranyl Acetate,"[(2E)-3,7-dimethylocta-2,6-dienyl] acetate",taste threshold values: taste characteristics ...,"odor of lavender@sweet, fruity-floral, rosy, s...",waxy@rose@lavender@green@floral@bitter


In [25]:
## count the freq of category of ingredients for each food
category_freq = merged_df[["id", "food", "ingredient", "category"]].drop_duplicates(subset=["id", "ingredient"]).groupby(['id', 'food','category'],  as_index=False)['id'].agg({'Frequency':'count'})

In [26]:
category_freq.info()
category_freq.iloc[:10, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4389 entries, 0 to 4388
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         4389 non-null   object
 1   food       4389 non-null   object
 2   category   4389 non-null   object
 3   Frequency  4389 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 137.3+ KB


,id,food,category,Frequency
0,005e671ac4,Pecan Pralines,additive,1
1,005e671ac4,Pecan Pralines,dairy,2
2,005e671ac4,Pecan Pralines,fruit-essence,1
3,005e671ac4,Pecan Pralines,nutseed-nut,1
4,00703db954,Greatest Granola,additive,1
5,00703db954,Greatest Granola,cerealcrop-cereal,1
6,00703db954,Greatest Granola,flower,1
7,00703db954,Greatest Granola,fruit,1
8,00703db954,Greatest Granola,nutseed-seed,1
9,00703db954,Greatest Granola,plant,1


In [27]:
## group the category freq into {key, val} pairs
category_freq["count"] = category_freq.apply(lambda x : {x["category"]: x["Frequency"]}, axis=1)
category_freq = category_freq.groupby(["id"], as_index=False).agg(category_freq=("count", lambda x : list(x)))

In [28]:
category_freq.iloc[:5, :]

,id,category_freq
0,005e671ac4,"[{'additive': 1}, {'dairy': 2}, {'fruit-essenc..."
1,00703db954,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'..."
2,007e544c80,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'..."
3,00a0ea8691,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'..."
4,00deb3f9f4,"[{'additive': 1}, {'fruit-berry': 1}, {'nutsee..."


In [29]:
## make set of molecules for each food
unique_molecules = merged_df[["id", "food", "ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name", "taste", "odor", "flavor_profile"]]
unique_molecules = unique_molecules.drop_duplicates(subset=["id", "pubchem_id"])

In [30]:
display(unique_molecules.iloc[:10, :])
unique_molecules.info()

,id,food,ingredient_flavorDB,fooddb_id,pubchem_id,common_name,taste,odor,flavor_profile
0,005e671ac4,Pecan Pralines,Vanilla,FDB009020,247,betaine,sweet,,bland
1,005e671ac4,Pecan Pralines,Vanilla,FDB003285,11552,3-Methylbutanal,"at very low levels the flavor is warm, herbace...","apple-like odor@powerful penetrating, acrid odor",peach@sour@chocolate@ethereal@malt@fatty@aldeh...
2,005e671ac4,Pecan Pralines,Butter,FDB012552,7501,Styrene,,"extremely penetrating@an aromatic odor@sweet, ...",balsamic@gasoline@floral@sweet@balsam@plastic
3,005e671ac4,Pecan Pralines,Vanilla,FDB020874,9895,beta-Cyclocitral,,,mint@saffron@tobacco@herbal@damascone@sweet@tr...
4,005e671ac4,Pecan Pralines,Butter,FDB001224,644104,CID 644104,,,grassy@very mild
5,005e671ac4,Pecan Pralines,Vanilla,FDB012090,7519,Anisole,sweet,sweet anise-like odor@agreeable aromatic odor@...,butter@ethereal@alcohol@gasoline@anise@cheese@...
6,005e671ac4,Pecan Pralines,Vanilla,FDB008173,11508,3-Methyl-1-Pentanol,,,wine@fusel@cocoa@fruity@cognac@green
7,005e671ac4,Pecan Pralines,Vanilla,FDB000881,13144,calcium lactate,,,odorless
8,005e671ac4,Pecan Pralines,Butter,FDB012084,12180,Methyl butyrate,"apple-like, sweet taste@not very powerful@belo...",apple-like odor,apple@pineapple@ether@banana@fruit@sweet@fruity
9,005e671ac4,Pecan Pralines,Vanilla,FDB013794,1549026,Geranyl Acetate,taste threshold values: taste characteristics ...,"odor of lavender@sweet, fruity-floral, rosy, s...",waxy@rose@lavender@green@floral@bitter


<class 'pandas.core.frame.DataFrame'>
Int64Index: 311515 entries, 0 to 584127
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   id                   311515 non-null  object
 1   food                 311515 non-null  object
 2   ingredient_flavorDB  311515 non-null  object
 3   fooddb_id            311515 non-null  object
 4   pubchem_id           311515 non-null  int64 
 5   common_name          311515 non-null  object
 6   taste                311515 non-null  object
 7   odor                 311515 non-null  object
 8   flavor_profile       311515 non-null  object
dtypes: int64(1), object(8)
memory usage: 23.8+ MB


In [31]:
## make molecule attributes into {key: val} pair
columns = ["ingredient_flavorDB", "fooddb_id", "pubchem_id", "common_name", "taste", "odor", "flavor_profile"]
unique_molecules["molecules"] = unique_molecules[columns].to_dict(orient="records")

## group by each food
unique_molecules = unique_molecules[["id", "food", "molecules"]]
unique_molecules = unique_molecules.groupby(["id", "food"], as_index=False).agg(flavor_molecules=("molecules", lambda x : list(x)))


In [32]:
display(unique_molecules)

,id,food,flavor_molecules
0,005e671ac4,Pecan Pralines,"[{'ingredient_flavorDB': 'Vanilla', 'fooddb_id..."
1,00703db954,Greatest Granola,"[{'ingredient_flavorDB': 'Sesame', 'fooddb_id'..."
2,007e544c80,Cream of Fresh Cauliflower Soup,"[{'ingredient_flavorDB': 'Milk', 'fooddb_id': ..."
3,00a0ea8691,Sausage and Rice Bake,"[{'ingredient_flavorDB': 'Rice', 'fooddb_id': ..."
4,00deb3f9f4,Spicy Raspberry Balsamic Dressing,"[{'ingredient_flavorDB': 'Raspberry', 'fooddb_..."
...,...,...,...
995,ff3633834a,Pumpkin Pie Oatmeal,"[{'ingredient_flavorDB': 'Pumpkin', 'fooddb_id..."
996,ff89c7f54c,Pizza Dough Focaccia (Abm),"[{'ingredient_flavorDB': 'Olive', 'fooddb_id':..."
997,ffb3c891c2,Cardinal Punch,"[{'ingredient_flavorDB': 'Lemon', 'fooddb_id':..."
998,ffde39a0da,Five Minute Berry Ice Cream,"[{'ingredient_flavorDB': 'Vanilla', 'fooddb_id..."


In [33]:
## count number of distinct flavor_molecules for each food
unique_molecules["molecules_count"] = unique_molecules.apply(lambda x : len(x['flavor_molecules']), axis=1)

In [34]:
display(unique_molecules.iloc[0:5, :])
unique_molecules.info()

,id,food,flavor_molecules,molecules_count
0,005e671ac4,Pecan Pralines,"[{'ingredient_flavorDB': 'Vanilla', 'fooddb_id...",261
1,00703db954,Greatest Granola,"[{'ingredient_flavorDB': 'Sesame', 'fooddb_id'...",306
2,007e544c80,Cream of Fresh Cauliflower Soup,"[{'ingredient_flavorDB': 'Milk', 'fooddb_id': ...",506
3,00a0ea8691,Sausage and Rice Bake,"[{'ingredient_flavorDB': 'Rice', 'fooddb_id': ...",470
4,00deb3f9f4,Spicy Raspberry Balsamic Dressing,"[{'ingredient_flavorDB': 'Raspberry', 'fooddb_...",462


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1000 non-null   object
 1   food              1000 non-null   object
 2   flavor_molecules  1000 non-null   object
 3   molecules_count   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [35]:
# add attributes: fsa_lights_per100g, ingredients, nutr_values_per100g
final_df = unique_molecules.merge(food_df[["id","instructions", "nutr_values_per100g", "fsa_lights_per100g"]], on="id", how="inner")

## add unique ingredients for each food
final_df = final_df.merge(unique_ingredient, on="id", how="left")
# add category freq
final_df = final_df.merge(category_freq, on="id", how="left")

# reorder column
final_df = final_df[["id", "food",  "unique_ingredient",  "ingredient_count", "category_freq", "instructions", "nutr_values_per100g", "fsa_lights_per100g", "flavor_molecules", "molecules_count"]]

In [36]:
final_df.info()
display(final_df.iloc[:10, :])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1000 non-null   object
 1   food                 1000 non-null   object
 2   unique_ingredient    1000 non-null   object
 3   ingredient_count     1000 non-null   int64 
 4   category_freq        1000 non-null   object
 5   instructions         1000 non-null   object
 6   nutr_values_per100g  1000 non-null   object
 7   fsa_lights_per100g   1000 non-null   object
 8   flavor_molecules     1000 non-null   object
 9   molecules_count      1000 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 85.9+ KB


,id,food,unique_ingredient,ingredient_count,category_freq,instructions,nutr_values_per100g,fsa_lights_per100g,flavor_molecules,molecules_count
0,005e671ac4,Pecan Pralines,"[cream, fluid, heavy whipping, sugars, brown, ...",5,"[{'additive': 1}, {'dairy': 2}, {'fruit-essenc...",[{'text': 'Butter a medium saucepan--this will...,"{'energy': 435.96474779673724, 'fat': 24.98968...","{'fat': 'red', 'salt': 'green', 'saturates': '...","[{'ingredient_flavorDB': 'Vanilla', 'fooddb_id...",261
1,00703db954,Greatest Granola,"[oats, seeds, sesame seeds, whole, dried, seed...",8,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'...","[{'text': 'Preheat oven to 300F.'}, {'text': '...","{'energy': 580.1557272750255, 'fat': 48.485788...","{'fat': 'red', 'salt': 'green', 'saturates': '...","[{'ingredient_flavorDB': 'Sesame', 'fooddb_id'...",306
2,007e544c80,Cream of Fresh Cauliflower Soup,"[soup, chicken broth or bouillon, dry, onions,...",9,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'...","[{'text': 'In a saucepan, combine the chicken ...","{'energy': 67.29209889741522, 'fat': 2.1389872...","{'fat': 'green', 'salt': 'red', 'saturates': '...","[{'ingredient_flavorDB': 'Milk', 'fooddb_id': ...",506
3,00a0ea8691,Sausage and Rice Bake,"[rice, white, long-grain, regular, unenriched,...",9,"[{'additive': 1}, {'cerealcrop-cereal': 1}, {'...","[{'text': 'In a shallow 2 qt casserole, place ...","{'energy': 123.63363927171838, 'fat': 10.13025...","{'fat': 'orange', 'salt': 'red', 'saturates': ...","[{'ingredient_flavorDB': 'Rice', 'fooddb_id': ...",470
4,00deb3f9f4,Spicy Raspberry Balsamic Dressing,"[oil, olive, salad or cooking, vinegar, balsam...",9,"[{'additive': 1}, {'fruit-berry': 1}, {'nutsee...","[{'text': 'Whisk olive oil, balsamic vinegar, ...","{'energy': 569.6636925188744, 'fat': 60.804891...","{'fat': 'red', 'salt': 'orange', 'saturates': ...","[{'ingredient_flavorDB': 'Raspberry', 'fooddb_...",462
5,00fb62a5c5,Roasted Strawberries with Black Pepper,"[strawberries, raw, sugars, granulated, spices...",4,"[{'additive': 1}, {'fruit-berry': 1}, {'plantd...",[{'text': 'Rinse the strawberries in cool wate...,"{'energy': 56.37505476851177, 'fat': 0.2994011...","{'fat': 'green', 'salt': 'green', 'saturates':...","[{'ingredient_flavorDB': 'Strawberry', 'fooddb...",382
6,01680dde21,Easy Cheese and Onion Bread,"[wheat flour, white, all-purpose, unenriched, ...",9,"[{'additive': 4}, {'cerealcrop-cereal': 1}, {'...","[{'text': 'Pre-heat oven to 375 degrees.'}, {'...","{'energy': 313.06332866624007, 'fat': 22.63932...","{'fat': 'red', 'salt': 'green', 'saturates': '...","[{'ingredient_flavorDB': 'Milk', 'fooddb_id': ...",372
7,01e8a5eafa,Orange Marshmallow Fruit Dip,"[cheese, parmesan, hard, candies, marshmallows...",4,"[{'bakery': 1}, {'beverage-alcoholic': 1}, {'d...",[{'text': 'Blend all ingredients until smooth....,"{'energy': 336.24522884980854, 'fat': 12.93647...","{'fat': 'orange', 'salt': 'red', 'saturates': ...","[{'ingredient_flavorDB': 'Parmesan Cheese', 'f...",357
8,021fc34777,Cherry Tomatoes (Stuffed) Recipe,"[cheese, cottage, creamed, large or small curd...",3,"[{'dairy': 1}, {'herb': 1}, {'vegetable': 1}]",[{'text': 'Scoop inside out of tomatoes with a...,"{'energy': 96.25225259952394, 'fat': 4.1964363...","{'fat': 'orange', 'salt': 'orange', 'saturates...","[{'ingredient_flavorDB': 'Dill', 'fooddb_id': ...",306
9,02ac814cc8,BBQ Short Ribs,"[pork, fresh, loin, country-style ribs, separa...",10,"[{'additive': 3}, {'cerealcrop-cereal': 1}, {'...","[{'text': 'Trim excess fat from ribs.'}, {'tex...","{'energy': 73.60967983814265, 'fat': 2.2422616...","{'fat': 'green', 'salt': 'green', 'saturates':...","[{'ingredient_flavorDB': 'Pork', 'fooddb_id': ...",550


In [37]:
## verify the freq of ingredient category, sum of freq should be equal to number of ingredients
final_df['category_freq'].apply(lambda x : sum(list(e.values())[0] for e in x))
final_df[final_df["ingredient_count"] != final_df['category_freq'].apply(lambda x : sum(list(e.values())[0] for e in x))].shape[0]

0

In [38]:
## small portion to view
final_df.iloc[:10, :].to_json(r'ugly_schema.json', orient="records")
!cat ugly.json | python -mjson.tool > schema.json
print("writing finished")

writing finished


In [36]:
## small portion to view
final_df.to_json(r'joined_dev_set.json', orient="records")
print("writing finished")

writing finished


In [37]:
!cat joined_dev_set.json | python -mjson.tool > pretty_joined_dev_set.json